In [2]:
import random
from datasets import load_dataset

In [3]:
dataset = load_dataset('Qwen/ProcessBench', split='olympiadbench')

In [16]:
with open("./resources/prompt_templates/DEPENDENCY_TRACKING_ONE.txt") as f:
    DEPENDENCY_TRACKING_PROMPT = f.read()

In [18]:
dataset[0]

{'id': 'olympiadbench-0',
 'generator': 'Llama-3.1-70B-Instruct',
 'problem': 'Find the smallest number $n$ such that there exist polynomials $f_{1}, f_{2}, \\ldots, f_{n}$ with rational coefficients satisfying\n\n$$\nx^{2}+7=f_{1}(x)^{2}+f_{2}(x)^{2}+\\cdots+f_{n}(x)^{2} .\n$$',
 'steps': ['To find the smallest number \\( n \\), we start by considering the given equation: \\( x^2 + 7 = f_1(x)^2 + f_2(x)^2 + \\cdots + f_n(x)^2 \\). Notice that \\( x^2 + 7 \\) is always greater than or equal to 7 for any real value of \\( x \\). We also observe that the right-hand side of the equation is a sum of squares of polynomials with rational coefficients. This means each term on the right-hand side must be non-negative.',
  'Now consider the case when \\( x = 0 \\). Plugging in \\( x = 0 \\) into the equation, we get: \\( 0^2 + 7 = f_1(0)^2 + f_2(0)^2 + \\cdots + f_n(0)^2 \\). This simplifies to \\( 7 = f_1(0)^2 + f_2(0)^2 + \\cdots + f_n(0)^2 \\). Since 7 is a prime number, the only way to expr

In [20]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 42
print(f"Sample Id: {sample_id}")
print("="*10)
print("Label:", dataset[sample_id]["label"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 42
Label: 1
As an expert reasoning graph architect, your task is to identify **direct dependencies** for **only the final step** in a given problem and solution by adhering to the following rules: **1. Direct Use Only**: Include a prior step in the final step's dependencies only if the final step explicitly relies on reasoning from that step, ignoring indirect dependencies. **2. Minimal & Non-Redundant**: If the final step depends on identical reasoning from multiple steps (e.g., the same lemma), include only the earliest step (smallest index), ensuring the set is the **minimal** required to justify the final step. **3. Output Format**: For the final step, identify any results, lemmas, or outputs from previous steps that inform its reasoning, marking them with `<expression>...</expression>` and specifying the source step with `<step>...</step>`. Provide a concise explanation of the final step's dependencies in a single short sentence. Do **not** include the results or output

In [9]:
sample_id = random.choice(range(len(dataset)))
step_id = random.choice(range(1, len(dataset[sample_id]["steps"])))
print(f"Sample Id: {sample_id}")
print(f"Step Id: {step_id}")
print("="*10)
print(dataset[sample_id]["problem"])
print("="*10)
print(dataset[sample_id]["steps"][step_id])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_previous_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"][:step_id])])
tagged_current_step = f"[{step_id}] {dataset[sample_id]['steps'][step_id]}"
print(DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_previous_steps=tagged_previous_steps, tagged_current_step=tagged_current_step
))

Sample Id: 527
Step Id: 7
Square $K E N T$ has side length 20 . Point $M$ lies in the interior of $K E N T$ such that $\triangle M E N$ is equilateral. Given that $K M^{2}=a-b \sqrt{3}$, where $a$ and $b$ are integers, compute $b$.
Substituting the value of \( x^2 \) from the previous step, we get \( KM^2 = 20^2 + \frac{1600 - 1200\sqrt{3}}{3} \).
You are an expert in logical dependency analysis for step-by-step problem solving.

**Task: **
Given a problem statement, a current step t, and a window of k previous steps, identify the **direct dependencies** of step t by outputting the indexes of the previous steps that step t **explicitly and necessarily relies on**. Only include steps whose results or lemmas are directly used in step t such that, if those steps are correct, step t can be verified independently.

**Input Format:**
- **Problem:** [Problem description]
- **Previous Steps:**
  - [index-1] [Step description]
  . . .
  - [index-k] [Step description]
- **Current Step:**
  - [in

In [8]:
# sample_id = random.choice(range(len(dataset)))
# step_id = random.choice(range(len(dataset[sample_id]["steps"])))
# print(f"Sample Id: {sample_id}")
# print(f"Step Id: {step_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
# print("="*10)
# print(dataset[sample_id]["steps"][step_id])
# print("="*10)
# print(STATE_EXTRACTING_PROMPT.format(problem=dataset[sample_id]["problem"], step=dataset[sample_id]["steps"][step_id]))

In [4]:
import sympy

In [5]:
sympy.parse_expr??

Signature:
sympy.parse_expr(
    s: str,
    local_dict: Optional[Dict[str, Any]] = None,
    transformations: Union[Tuple[Callable[[List[Tuple[int, str]], Dict[str, Any], Dict[str, Any]], List[Tuple[int, str]]], ...], str] = (<function lambda_notation at 0x0000027F97E1CC20>, <function auto_symbol at 0x0000027F97E1CB80>, <function repeated_decimals at 0x0000027F97E1CE00>, <function auto_number at 0x0000027F97E1CEA0>, <function factorial_notation at 0x0000027F97E1CCC0>),
    global_dict: Optional[Dict[str, Any]] = None,
    evaluate=True,
)
Source:   
def parse_expr(s: str, local_dict: Optional[DICT] = None,
               transformations: tUnion[tTuple[TRANS, ...], str] \
                   = standard_transformations,
               global_dict: Optional[DICT] = None, evaluate=True):
    """Converts the string ``s`` to a SymPy expression, in ``local_dict``.

    Parameters

    s : str
        The string to parse.

    local_dict : dict, optional
        A dictionary of local variables

In [9]:
from sympy.parsing.latex import parse_latex
latex_expr_1 = "x^2+x + 1 = 0"
latex_expr_2 = "1 + x^2 + x=0"
sympy_expr_1 = parse_latex(latex_expr_1)
sympy_expr_2 = parse_latex(latex_expr_2)
print(sympy_expr_1)
print(sympy_expr_2)
print(str(sympy_expr_1) == str(sympy_expr_2))


Eq((x**2 + x) + 1, 0)
Eq(x + (x**2 + 1), 0)
False


In [2]:
sympy_expr

Eq(x**2 - 1, 0)

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [37]:
from transformers import AutoTokenizer, GenerationConfig

from tqdm import tqdm

# model_name = "Qwen/Qwen3-14B"
model_name = "Qwen/Qwen3-32B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)

generation_config = GenerationConfig.from_pretrained(model_name)

def is_chinese(token: str) -> bool:
    return any(0x4E00 <= ord(c) <= 0x9FFF or 0x3400 <= ord(c) <= 0x4DBF or 0xF900 <= ord(c) <= 0xFAFF for c in token)

print("is_chinese('该助手为DeepSeek'): ", is_chinese("该助手为DeepSeek"))

ban_ids = []
for k, v in tqdm(tokenizer.vocab.items()):
    if is_chinese(tokenizer.decode([v])):
        ban_ids.append(v)

print("len(ban_ids): ", len(ban_ids))
print("ban_ids: ", ban_ids)

generation_config.suppress_tokens = ban_ids
generation_config.save_pretrained("./tmp")

# When deploy model with vllm, add the following command: --generation-config auto

is_chinese('该助手为DeepSeek'):  True


100%|██████████| 151669/151669 [00:01<00:00, 89950.19it/s]

len(ban_ids):  25860
ban_ids:  [101565, 114016, 100065, 100426, 115379, 104677, 104999, 110165, 108676, 107318, 121187, 116483, 123242, 121251, 108935, 103651, 110244, 111583, 109024, 106063, 108390, 120894, 100898, 120503, 101491, 108133, 101780, 104546, 108851, 122667, 99371, 118402, 60533, 120376, 118788, 105162, 108288, 112697, 103295, 116066, 109602, 103341, 105628, 103609, 120265, 105381, 114381, 121607, 109922, 114460, 101848, 105327, 100798, 121074, 115519, 122354, 115476, 123422, 111232, 114100, 118503, 99925, 122356, 115954, 116249, 118582, 116207, 120709, 109652, 117315, 104498, 106698, 109397, 103969, 119742, 107050, 100508, 117567, 117486, 115498, 114540, 115046, 99438, 111194, 107973, 111802, 104487, 102066, 118600, 107411, 107556, 115896, 104596, 106069, 114390, 114423, 121804, 105466, 107726, 102060, 102425, 120970, 102565, 112382, 108421, 100770, 106945, 106941, 99641, 110174, 142808, 147596, 102147, 105731, 120327, 109819, 106834, 123457, 104967, 113223, 120652, 11865

In [35]:
tokenizer.decode(119325)

'拄'

In [31]:
generation_config.suppress_tokens = ban_ids

In [12]:
is_chinese("该助手为DeepSeek")

True

In [6]:
tokenizer.batch_decode([19768])

['.Media']

In [7]:
# prepare the model input
systemp_prompt = """
该助手为DeepSeek Chat，由深度求索公司创造。
今天是3月24日，星期一。
""".strip()
messages = [
    {"role": "system", "content": systemp_prompt},
] if systemp_prompt else []
messages.append({"role": "user", "content": prompt})
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
print(text)

<｜begin▁of▁sentence｜>该助手为DeepSeek Chat，由深度求索公司创造。
今天是3月24日，星期一。<｜User｜>You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Solve for x: 2x + 5 = 15
Steps:
[0] Subtract 5 from both sides: 2x = 10
[1] Divide both sides by 2: x = 5
Analysis:
- Step 0: performs

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 

In [ ]:
# sample_id = random.choice(range(len(dataset)))
# while len(dataset[sample_id]["steps"]) <= 10:
#     sample_id = random.choice(range(len(dataset)))
sample_id = 873
print(f"Sample Id: {sample_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
prompt = DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_steps=tagged_steps
)
print(prompt)

Sample Id: 873
You are an expert reasoning graph architect. Given a problem and step-by-step solution, identify **direct dependencies** between steps using this format:

### Rules:
1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:
   - For each step `t`:
     - Identify ALL used result, lemma, or output with `<expression>...</expression>`
     - Specify source step with `<step>...</step>`

### Example:
Problem: Find circle area with radius 5  
Steps:  
[0] Recall formula: Area = π * r²  
[1] Plug in radius: Area = π * 5²  
[2] Calculate 5²: 25  
[3] Multiply: Area = 25π  

Analysis:  
- Step 